In [ ]:
import sqlite3
import re
import tqdm
con = sqlite3.connect("db/main.db")
cur = con.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS "Log_table" (
	"id"	INTEGER,
	"date"	TEXT,
	"text"	TEXT,
	"label"	TEXT,
	"info"	TEXT,
	PRIMARY KEY("id")
);""")
con.commit()
cur.execute("""CREATE TABLE IF NOT EXISTS "Sample_table" (
	"id"	INTEGER,
	"text_en"	TEXT,
	"text_ru"	TEXT DEFAULT NULL,
	"label"	TEXT,
	"processed"	INTEGER DEFAULT 0,
	PRIMARY KEY("id")
);""")
con.commit()
cur.execute("""Delete from "Sample_table";""")
con.commit()

for i in range(10):
    con_old = sqlite3.connect(f"homework/data/db/mydatabase_{i}.db")
    cur_old = con_old.cursor()

    res = cur_old.execute("Select * from emotions").fetchall()
    for item in res:
        cur.execute("INSERT INTO \"Sample_table\" (text_en, text_ru, label, processed) VALUES (?, ?, ?, ?)",
                              (item[2], item[3], item[1], 0))

    con_old.commit()
    con_old.close()
    con.commit()
    
cur.execute("""CREATE TABLE IF NOT EXISTS "Usage_table" (
	"id"	INTEGER,
	"word"	TEXT NOT NULL,
	"language"	TEXT NOT NULL,
	"label"	TEXT NOT NULL,
	"usage"	INTEGER NOT NULL DEFAULT 0,
	PRIMARY KEY("id")
);""")
con.commit()

# import pymorphy2
# from functools import lru_cache

# @lru_cache(maxsize=None)
# def generate_cases(word, morph):
#     parsed_word = morph.parse(word)[0]
#     try:
#         cases = {
#             'Именительный': parsed_word.inflect({'nomn'}).word or word,
#             'Родительный': parsed_word.inflect({'gent'}).word or word,
#             'Дательный': parsed_word.inflect({'datv'}).word or word,
#             'Винительный': parsed_word.inflect({'accs'}).word or word,
#             'Творительный': parsed_word.inflect({'ablt'}).word or word,
#             'Предложный': parsed_word.inflect({'loct'}).word or word,
#         }
#     except:
#         cases = {"Основа" : word}
#     return cases



cur.execute("""Delete from "Usage_table";""")
con.commit()

# res = cur.execute("""
# Select * from "Sample_table" where "text_ru" is not NULL limit 5;
# """).fetchall()

res = cur.execute("""
Select * from "Sample_table" where "text_ru" is not NULL;
""").fetchall()

s = []
for i in cur.execute('select label from Sample_table where text_ru is not NULL').fetchall():
    if i[0] not in s:
        s.append(i[0])

all_usage = {}
for i in s:
	all_usage[i] = {}

for row in tqdm(res, total=len(res)):
	label = row[3] 
	for word in row[1].split():
		new_word = re.sub(r"[^а-яА-ЯёЁ]", r"", word.lower())
		if new_word != '':
			# print(new_word)
			# cases = generate_cases(new_word, morph)
			word_ = new_word
			if word_ in all_usage[label]:
				all_usage[label][word_] += 1
			else:
				all_usage[label][word_] = 1
	

for label in all_usage:
	for word in all_usage[label]:
		new_word = word
		count = all_usage[label][word]
		cur.execute("insert into \"Usage_table\" (word, language, label, usage) values (?, ?, ?, ?)", (new_word, "ru", label, str(count)))
con.commit()

TypeError: 'module' object is not callable